In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import category_encoders as ce
import warnings
warnings.filterwarnings("ignore")

In [ ]:
grades_df = pd.read_csv('school_grades_dataset.csv')


with pd.option_context('display.max_columns', None):
    print(grades_df.head())

In [ ]:
grades_df.info()

In [ ]:
attributes_df = pd.read_csv('attributes_school_grades.csv')

with pd.option_context('max_colwidth', 150):
    print(attributes_df)

In [ ]:
grades_df.describe()

In [ ]:
# Mamy bardzo dużo zmiennych kategorycznych. W danych nie ma braków. Spróbujmy zakodować zmienne kategorczne:

# Zmienne: schoolsup, famsup, paid, activities, nursery, higher, internet, romantic jako zmienne o wartościach yes/no możemy
# zakodować za pomocą Ordinal Encoder'a
cols = ['schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']
grades_df_new = grades_df.drop(columns = cols)


for i in cols:
    encoder = ce.OrdinalEncoder(mapping = [{'col': i, 'mapping': {'yes': 1, 'no': 0}},])
    grades_df_new[i] = encoder.fit_transform(grades_df)[i]
    

bin_cols = cols + ['school', 'sex', 'address', 'famsize', 'Pstatus']

In [ ]:
sns.pairplot(grades_df_new, y_vars="G3", x_vars=grades_df.columns.values[0:5])
sns.pairplot(grades_df_new, y_vars="G3", x_vars=grades_df.columns.values[5:10])
sns.pairplot(grades_df_new, y_vars="G3", x_vars=grades_df.columns.values[10:15])
sns.pairplot(grades_df_new, y_vars="G3", x_vars=grades_df.columns.values[15:20])
sns.pairplot(grades_df_new, y_vars="G3", x_vars=grades_df.columns.values[20:25])
sns.pairplot(grades_df_new, y_vars="G3", x_vars=grades_df.columns.values[25:30])
sns.pairplot(grades_df_new, y_vars="G3", x_vars=grades_df.columns.values[30:])

plt.show()

In [ ]:
corr = grades_df_new.corr()
f, ax = plt.subplots(figsize=(20, 12))
ax = sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,
            annot = True,
            fmt='.2f')

In [ ]:
#heatmapa tylko dla zminnych numerycznych

fig, ax = plt.subplots(figsize=(13, 13))
sns.heatmap(grades_df.corr(), annot=True, square=True,  fmt='.2f')
plt.show()

In [ ]:
cols = grades_df.describe().columns

grades_df[cols].hist(bins = 20, figsize=(21, 15))
plt.show()

In [ ]:
discrete_vals = grades_df.drop(cols, axis=1)

len(discrete_vals.columns)

def y_or_n_vals(col):
    if len(col.value_counts()) == 2:
        if all(col.value_counts().index == ['yes', 'no']) | all(col.value_counts().index == ['no', 'yes']):
            return True
    return False

fig, axs = plt.subplots(6, 3, figsize = (18, 25))
for i in range(len(discrete_vals.columns)):
    col = discrete_vals.columns[i]
    
    if y_or_n_vals(discrete_vals[col]):
        sns.countplot(data = discrete_vals, x=col, ax = axs[i // 3, i % 3], order = ['yes', 'no'])
    else:
        order = discrete_vals[col].value_counts().sort_values(ascending=False).index
        sns.countplot(data = discrete_vals, x=col, ax = axs[i // 3, i % 3], order = order)
plt.show()

In [ ]:
df = grades_df.drop(['G1', 'G2', 'absences'], axis = 1)

def one_to_five_vals():
    if len(col.value_counts()) == 5:
        if all(col.value_counts().index == ['yes', 'no']) | all(col.value_counts().index == ['no', 'yes']):
            return True
    return False

fig, axs = plt.subplots(10, 3, figsize = (18, 40))
for i in range(len(df.columns)-1):
    col = df.columns[i]
    
    if y_or_n_vals(df[col]):
        sns.violinplot(data = df, x=col, y = 'G3', ax = axs[i // 3, i % 3], order = ['yes', 'no'])
    elif type(df[col].value_counts().index[0]) is np.int64:
        sns.violinplot(data = df, x=col, y = 'G3', ax = axs[i // 3, i % 3], order = df[col].unique().sort())
    else:
        order = df[col].value_counts().sort_values(ascending=False).index
        sns.violinplot(data = df, x=col, y = 'G3', ax = axs[i // 3, i % 3], order = order)
plt.show()

In [ ]:
#Jeszcze raz te same wykresy, żeby wstawić do prezentacji

#fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(14, 4))
#sns.violinplot(data = grades_df, x='nursery', y = 'G3', ax = ax1, order = ['yes', 'no'])
#sns.violinplot(data = grades_df, x='reason', y = 'G3', ax = ax2, order = grades_df['reason'].unique())
#sns.violinplot(data = grades_df, x='higher', y = 'G3', ax = ax3, order = ['yes', 'no'])
#plt.show()


In [ ]:
fig, axs = plt.subplots(len(bin_cols), 2, figsize = (20, 65))
for i in range(len(bin_cols)):
    a, b = sorted(grades_df[bin_cols[i]].unique())
    if (a == 'no'): a,b = b,a
    
    X = grades_df.loc[grades_df[bin_cols[i]] == a]
    plot_dens=sns.distplot(X['G3'], ax = axs[i, 0])
    plot_dens.set_title('Rozkład zmiennej G3 dla zmiennej ' + bin_cols[i] + ' równe ' + a)
    txt = "mean: " + str(np.mean(X['G3']).round(2))
    top = (max(X['G3'].value_counts()))/len(X)
    plot_dens.text(0, 0.75 * top,s = txt, fontsize=15)
    txt = "std: " + str(np.std(X['G3']).round(2))
    plot_dens.text(0, 0.6 * top,s = txt, fontsize=15)
    
    X = grades_df.loc[grades_df[bin_cols[i]] == b]
    plot_dens=sns.distplot(X['G3'], ax = axs[i, 1])
    plot_dens.set_title('Rozkład zmiennej G3 dla zmiennej ' + bin_cols[i] + ' równej ' + b)
    txt = "średnia: " + str(np.mean(X['G3']).round(2))
    top = (max(X['G3'].value_counts()))/len(X)
    plot_dens.text(0, 0.85 * top,s= txt, fontsize=15)
    txt = "std: " + str(np.std(X['G3']).round(2))
    plot_dens.text(0, 0.7 * top,s = txt, fontsize=15)
plt.show()

- Osoby nie posiadający korepetycji uczą się lepiej (zaskakujące). 
- Osoby posiadające wsparcie rodziny uczą się lepiej. Wykresy osób posiadających / nie posiadających korepetycji przypominają rozkład normalny.
- Osoby biorące udział w zajęciach pozalekcyjnych lepiej się uczą. 
- Osoby po przedszkolu lepiej się uczą. 
- Zdecydowanie lepiej uczą się osoby chcące osiągnąć wykształcenie wyższe 
- Zdecydowanie lepiej uczą się osoby posiadające dostęp do internetu
- Rozkłady zmiennej romantic wyglądają podobnie, natomiast znacznie wyższa średnia wychodzi gdy 'romantic' jest 'no' Lepsze oceny mają uczniowie szkoły GP. 
- Dziewczynki uczą się lepiej od chłopców. 
- Mieszkańcy miast uczą się lepiej niż mieszkańcy wsi. 
- Histogram sugeruje, że lepiej uczą się dzieci z małych rodzin, matomiast wyższą średnią ocen uzyskały dzieci z dużych rodzin. - Rozkłady dla zmiennej "Pstatus" znacząco się różnią, natomiast średnia wygląda identycznie.
- szkoła GP lepsza od MS

In [ ]:
cols_cat = ['Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'traveltime', 'studytime', 'failures']

for i in range(len(cols_cat)):
    vals = sorted(grades_df[cols_cat[i]].unique())
    
    fig, axs = plt.subplots(1, len(vals), figsize = (20, 4))
    for a in range(len(vals)):
        X = grades_df.loc[grades_df[cols_cat[i]] == vals[a]]
        
        plot_dens=sns.distplot(X['G3'], ax = axs[a])
        plot_dens.set_title(cols_cat[i] + ' = ' + str(vals[a]))
        

    plt.show()

In [ ]:
grades_df['Mjob'].value_counts()

In [ ]:
grades_df['Fjob'].value_counts()

- Najlepiej uczą się osoby, dla których współczynnik Medu i Fedu jest większy od 2, co było do przewidzenia, że dzieci wykształconych rodziców będą miały dobre oceny.
- Jeśli chodzi o wykształcenie rodziców, to najwyższe oceny mają dzieci lekarzy i nauczycieli, jednak te grupy są stosunkwo nieliczne w porównaniu do pozostałych
- Najlepsze oceny mają osoby, które wybrały szkołę ze względu na jej reputację, natomiast histogram dla powodu 'pozostałe' jest bardzo podobny do rozkładu naturalnego.
- Jeśli chodzi o wolny czas, to najlepiej uczą się osoby ze środka: Nadmiar / niedomiar wolnego czasu nie sprzyja dobrym ocenom.
- Pesymistycznym wnioskiem jest, że lepiej uczą się osoby rzadko wychodzące ze znajomymi.
- Spodziewanym wnioskiem jest, żę osoby pijące mało alkoholu, zarówno w dni powszednie, jak i weekendy mają lepsze oceny. 
- Osoby o lepszym stanie zdrowia, mają lepsze oceny.
- Lepiej uczą się osoby, które mało czasu przeznaczają na podróż do szkoły.
- Lepiej uczą się osoby, które wiecej czasu przeznaczaja na naukę
- Lepiej uczą się osoby nie mające na koncie porażek, histogram przypomina wykres rozkładu normalnego
- najbardziej skorelowane są wyniki egzaminów - uczniowie utrzymują stały poziom

In [ ]:
grades_df['G3-result'] = np.where(grades_df['G3'] < 10, 'failed',
                                 np.where(grades_df['G3'] < 13, 'weak',
                                 np.where(grades_df['G3'] < 16, 'ok',
                                 np.where(grades_df['G3'] < 19, 'good', 'excellent'))))

grades_df.head()


In [ ]:
grades_df['G3-result'].value_counts()

In [ ]:
interesting_cols = ['Medu', 'Fedu', 'traveltime', 'absences', 'Dalc', 'Walc', 'failures', 'G3-result']
df = grades_df[interesting_cols]
order = ['failed', 'weak', 'ok', 'good', 'excellent']
n = len(interesting_cols)
k = 0

fig, axs = plt.subplots(n, 1, figsize=(40, 80))
for i in range(n-1):
    col = interesting_cols[i]
    sns.swarmplot(data = grades_df, x = 'G3-result', y = col,  order = order, ax = axs[i])
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 8))
sns.swarmplot(data = grades_df, x = 'G3-result', y = 'absences',  order = order)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(18, 5))
sns.swarmplot(data = grades_df, x = 'G3-result', y = 'failures',  order = order)
plt.show()